# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to create personalized content such as social media posts, product descriptions, and blog articles, saving time and increasing efficiency for marketers.
2. **Product Design**: Generative AI can assist in designing products by generating prototypes, testing their feasibility, and optimizing their design.
3. **Image and Video Generation**: Generative AI can be used to create high-quality images and videos for marketing campaigns, advertising, and social media content.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequent questions and helping customers with simple queries.
5. **Content Writing**: Generative AI can assist in writing articles, blog posts, and other written content, freeing up human writers to focus on more complex tasks.
6. **Language Translation**: Generative AI-powered t

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, product descriptions, and even entire books. This can help businesses reduce content creation costs and improve consistency.
2. **Product Design and Development**: Generative AI can assist in designing new products, such as furniture, cars, or electronics, by generating 3D models, prototypes, and user interfaces.
3. **Marketing and Advertising**: Generative AI can create personalized ad copy, product recommendations, and social media ads, improving campaign engagement and conversion rates.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, respond to common queries, and even offer personalized advice and solutions.
5. **Data Analysis and Visualization**: Generative AI can help analyze large datasets, identify patterns, and create visualizatio

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [7]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [12]:
summarize("https://edwarddonner.com")

'**Website Summary**\n=====================\n\n*   **Mission**: Apply AI to help people discover their potential and pursue their purpose.\n*   **Product**: Nebula.io, a platform that sources, understands, engages, and manages talent for recruiters.\n\n**Recent News/Announcements**\n-----------------------------\n\n### Mastering AI and LLM Engineering – Resources (Nov 13, 2024)\n\nResources available to learn about mastering AI and LLM engineering.\n\n### From Software Engineer to AI Data Scientist – resources (Oct 16, 2024)\n\nAdditional resources for those looking to transition from software engineer to AI data scientist.\n\n### Outsmart LLM Arena – a battle of diplomacy and deviousness (Jun 26, 2024)\n\nAn arena where large language models compete in a battle of wits.\n\n### Choosing the Right LLM: Toolkit and Resources (Aug 6, 2024)\n\nA collection of tools and resources for choosing the right large language model.'

In [14]:
print(summarize("https://google.com"))

### Google Website Summary

#### Overview
The website is the official homepage of Google, a multinational technology giant.

#### Featured Sections

*   **Services**: Offers various services including Search, Images, Maps, Play, YouTube, and more.
*   **Sign-in and Settings**: Allows users to sign in and access their account settings.
*   **Advanced Features**: Includes features like Web History and Advanced search.
*   **Language Support**: Available in multiple languages.
*   **Support Pages**: Links to Advertising, Business Solutions, About Google, and Privacy pages.

#### No News or Announcements Found
There are no news or announcements on the homepage.


In [17]:
print(summarize("https://www.facebook.com"))

**Facebook Website Summary**

### Overview

Facebook is a social networking platform that allows users to connect with others and share content.

### Features

* **Login/Sign-up**: Users can log in or sign up for an account.
* **Language Options**: The website offers multiple language options, including Hindi, Urdu, Punjabi, Bengali, Gujarati, Marathi, Tamil, Telugu, Malayalam, Kannada, and English (UK).
* **Other Services**: Facebook offers additional services such as Messenger, Facebook Lite, Video, Marketplace, Meta Pay, Meta Store, Meta Quest, Ray-Ban Meta, Meta AI, Instagram, Threads, Donations Center, and Voting Information Center.

### Security and Policies

* **Privacy Policy**: The website provides a privacy policy for users.
* **Privacy Center**: A dedicated page for managing user privacy settings.
* **Terms of Service**: Users must agree to the terms of service before using the platform.
* **Cookies**: The website uses cookies for advertising purposes.

### Developer Resourc

In [22]:
print(summarize("https://perplexity.ai"))

# Website Summary

## Description
Just a moment... is a website with no apparent content or purpose. The only message displayed is "Enable JavaScript and cookies to continue", suggesting that the website requires enabling JavaScript and cookie permissions in order to function properly.

## No News or Announcements Found
There are no news articles, announcements, or updates on this website at present.
